In [1]:
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from eli5.sklearn import PermutationImportance
from catboost import CatBoostRegressor,Pool
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import norm
from scipy import linalg
from sklearn import tree
from sklearn import svm
import lightgbm as lgb
#import xgboost as xgb
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import graphviz
import warnings
import random
import eli5
#import shap  # package used to calculate Shap values
import time
import glob
import sys
import os
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression

print("Import finishes")

/Users/wangrunqiu/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/wangrunqiu/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


Import finishes


In [3]:
warnings.filterwarnings("ignore")

print("Reading csv file...")
train = pd.read_csv('./train.csv' , dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})
print("Train: rows:{} columns:{}".format(train.shape[0], train.shape[1]))

Reading csv file...
Train: rows:629145480 columns:2


In [5]:
rows = 150_000
segments = int(np.floor(train.shape[0] / rows)) # 4194

X_train = pd.DataFrame(index=range(segments), dtype=np.float64) # 使用对应的特征

y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

submission = pd.read_csv('./sample_submission.csv', index_col='seg_id')

X_train =pd.read_csv('./data/4194samples_137/X_train_138features_filtered.csv',index_col=0)
X_test = pd.read_csv('./data/4194samples_137/X_test_138features_filtered.csv',index_col=0)

#y_train

scale = 1  # test:使滑动窗口缩小10倍
for segment in range(segments):
    # y_train and X_train
    #     if segment * rows / scale + rows > train.shape[0]:
    #         break
    seg = train.iloc[int(segment * rows / scale): int(segment * rows / scale + rows)]
    x = pd.Series(seg['acoustic_data'].values)
    y = seg['time_to_failure'].values[-1]
    y_train.loc[segment, 'time_to_failure'] = y
#print(X_train.head())
#print(X_test.head())
print(y_train.shape)
#print("submission has: rows:{} cols:{}".format(submission.shape[0], submission.shape[1]))
print("Reading ends.")


(4194, 1)
Reading ends.


In [6]:
# 正则化
print("FillNA and standardScalar...")
means_dict = {}
for col in X_train.columns:
    if X_train[col].isnull().any():
        print(col)
        mean_value = X_train.loc[X_train[col] != -np.inf, col].mean()
        X_train.loc[X_train[col] == -np.inf, col] = mean_value
        X_train[col] = X_train[col].fillna(mean_value)
        means_dict[col] = mean_value

for col in X_test.columns:
    if X_test[col].isnull().any():
        X_test.loc[X_test[col] == -np.inf, col] = means_dict[col]
        X_test[col] = X_test[col].fillna(means_dict[col])

X = X_train.copy()
y = y_train.copy()
#train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
print(X_train_scaled.shape)
print("FillNA and standardScalar ends.")

FillNA and standardScalar...
(4194, 137)
FillNA and standardScalar ends.


In [7]:
print('start')
from sklearn.model_selection import GridSearchCV
param_test2 = {'n_estimators':range(100,1000,100),'learning_rate':[0.01,0.02,0.03,0.04,0.05],'max_depth':range(1,5),'max_bin':range(50,300,10)}
gsearch2 = GridSearchCV(estimator = lgb.LGBMRegressor(), param_grid = param_test2, scoring=None,cv=5)
gsearch2.fit(X_train_scaled,y_train)
print(gsearch2.best_params_)
submission['time_to_failure'] = gsearch2.predict(X_test_scaled)
submission.to_csv('submission_lgb1.csv')
print('end')

start
{'learning_rate': 0.05, 'max_bin': 60, 'max_depth': 1, 'n_estimators': 100}
end
